# Spring 2019 CS 6400
## Structured Query Language (SQL)

Copyright: Travis Jefferies
Last Updated: 03/31/2019

### History

* Originally known as Structured English QUEry Language (SEQUEL)
    * Part of SYSTEM R, 1973
* Now known as SQL
    * Based on relational tuple calculus and some algebra
* ANSI & ISO Standards over the years
* Supported by almost all commercial RDMBS

### Insert, Delete, and Update

#### Insert

Given the relation:

**UserInterests**

| Email | Interest | SinceAge
| --- | --- | --- |
| user1@gatech.edu | Swimming | 7 |


To insert new rows into the  relation, we use the `INSERT` statement like so:

`INSERT INTO UserInterests (Email, Interest, SinceAge)
VALUES ('user12@gt.edu', 'Reading', 5)`

After the `INSERT` statement above...

**UserInterests**

| Email | Interest | SinceAge
| --- | --- | --- |
| user1@gatech.edu | Swimming | 7 |
| user2@gatech.edu | Reading | 5 |

More generally, the syntax for `INSERT` is as follows:

`INSERT INTO table (col_1, col_2, col_n)
VALUES (val_1, val_2, val_n)`

By default, `INSERT` only processes a single row at a time.<br>
Unless, we specify that the `INSERT` statement use values from a `SELECT` subquery.

#### Delete

Now let's say we want to delete certain rows from the **UserInterests** relation - we use the `DELETE` keyword for this:

`DELETE FROM UserInterests
WHERE Interest = 'Swimming'`

After execution, our **UserInterests** relation will look like:

**UserInterests**

| Email | Interest | SinceAge
| --- | --- | --- |
| user2@gatech.edu | Reading | 5 |

General form of `DELETE` statement:

`DELETE FROM table
WHERE col_n = val`

#### Update

We can use the `UPDATE` keyword to change existing values in a given relation:

`UPDATE UserInterests
SET Interest = 'Music'
WHERE Email = 'user2@gatech.edu'`

After the statement executes, the **UserInterests** relation will look like:

| Email | Interest | SinceAge
| --- | --- | --- |
| user2@gatech.edu | Music | 5 |

General form:

`UPDATE table
SET col_n = val
WHERE col_m = val_m`

### General SQL Query Syntax

Almost all SQL queries follow the same general syntax:

`SELECT col_1, col_2, col_n
FROM table_1, table_2, table_m
WHERE condition`

* column$_{\,n}$ is the name of a **column**, like BirthYear
* table$_{\,m}$ is the name of a **table**, like RegularUser
* **condition** may compare values of columns to constants or to each other.
    * BirthYear > 1985
    * CurrentCity = HomeTown
* conditions can be combined with **AND**, **OR**, **NOT** and **()**

In relational algebra, our general syntax becomes:

$\pi_{\text{column}_{1}, \ldots, {\text{column}_{n}}}\left(\sigma_{\text{condition}}\left(\text{table}_{1}\times \ldots \times \text{table}_{m}\right)\right)$

* all sql queries must have `SELECT` and `FROM` clauses.
* the sql query is equivalent to the relational algebra query (with some exceptions).
* if there is no `WHERE` clause, the query defaults to a Cartesian product followed by the specified projection.

### Joins

#### Inner Join and Dot Notation

Given:

**RegularUser**

| Email | BirthYear | Sex
| --- | --- | --- |
| user1@gatech.edu | 1985 | M |
| user2@gatech.edu | 1969 | M |
| user3@gatech.edu | 1967 | M |
| user4@gatech.edu | 1988 | M |
| user6@gatech.edu | 1988 | F |
| user8@gatech.edu | 1968 | M |
| user9@gatech.edu | 1988 | F |

Given:

**YearSalary**

| BirthYear | Salary
| --- | --- |
| 1985 | 27000 |
| 1969 | 43000 |
| 1967 | 45000 |

The SQL statement below performs an inner join between **RegularUser** and **YearSalary**:

`SELECT Email, RegularUser.BirthYear, Salary
FROM RegularUser, YearSalary
WHERE RegularUser.BirthYear = YearSalary.BirthYear`

Result:

| Email | BirthYear | Salary
| --- | --- | --- |
| user1@gatech.edu | 1985 | 27000 |
| user2@gatech.edu | 1969 | 43000 |
| user3@gatech.edu | 1967 | 45000 |


Notice that we must use dot notation (table_m.col_n) to distinguish which relation to pull the `BirthYear` column from. This is required to resolve ambiguitity.

#### Natural Join

We get the exact same result set as above using a natural join statement:

`SELECT Email, RegularUser.BirthYear, Salary
FROM RegularUser NATURAL JOIN YearSalary`

Result:

| Email | BirthYear | Salary
| --- | --- | --- |
| user1@gatech.edu | 1985 | 27000 |
| user2@gatech.edu | 1969 | 43000 |
| user3@gatech.edu | 1967 | 45000 |

A `NATURAL JOIN` statement joins two relations on the columns that have the same names.

We still must use dot notation (table_m.col_n) to distinguish which relation to pull the `BirthYear` column from. This is required to resolve ambiguitity.

In the case where there are no columns that have the same names, the `NATURAL JOIN` statement defaults to the Cartesian Product. A `NATURAL JOIN` statement never returns duplicate column names!

#### Aliases

We can write the same query as above using aliases instead of the whole table name:

`SELECT Email, R.BirthYear, Salary
FROM RegularUser AS R, YearSalary Y
WHERE R.BirthYear = Y.BirthYear`

`R` and `Y` are tuple variables.<br>
* Tuple variables aka aliases save typing
* Used to disambiguate table references
* Aliases MUST be used when joining a table to itself

#### Left Outer Joins

To include results from one "left" of the relations without a resulting match in the second "right" relation, we can use a left join:

`SELECT Email, RegularUser.BirthYear, Salary
FROM RegularUser LEFT OUTER JOIN YearSalary`

Result:

| Email | BirthYear | Salary
| --- | --- | --- |
| user1@gatech.edu | 1985 | 27000 |
| user2@gatech.edu | 1969 | 43000 |
| user3@gatech.edu | 1967 | 45000 |
| user4@gatech.edu | 1988 | NULL |
| user6@gatech.edu | 1988 | NULL |
| user8@gatech.edu | 1968 | NULL |
| user9@gatech.edu | 1988 | NULL |

Note that since users 4,6,8, and 9 do not have a corresponding `YearSalary.BirthYear` record, their values for `Salary` have been filled with NULL.

### String Matching

SQL is a practical tool that needs certain capabilities outside the scope of relational algebra and relational calculus - string matching and the `DISTINCT` keywords being perfect examples.

Let's say our objective is to *find data about RegularUsers who live in a CurrentCity that starts with "San"*. We can use the string matching capability that is native to base SQL:

Given:

**RegularUser**

| Email | Sex | CurrentCity | HomeTown
| --- | --- | --- | --- |
| user1@gatech.edu | M | Seattle | Atlanta
| user3@gatech.edu | M | San Diego | Portland
| user4@gatech.edu | M | San Francisco | Atlanta
| user6@gatech.edu | F | San Diego | Atlanta
| user9@gatech.edu | F | Las Vegas | Atlanta
| user12@gatech.edu | F | College Park | Austin

SQL:

`SELECT Email, Sex, CurrentCity
FROM RegularUser
WHERE CurrentCity LIKE 'San%';`

Result: 

| Email | Sex | CurrentCity |
| --- | --- | --- |
| user3@gatech.edu | M | San Diego |
| user4@gatech.edu | M | San Francisco |
| user6@gatech.edu | F | San Diego |

We can also use `_` as a wildcard to match any single character:

SQL:

`SELECT Email, Sex, HomeTown
FROM RegularUser
WHERE HomeTown LIKE 'A_____';`

Result:

| Email | Sex  | HomeTown
| --- | ---  | --- |
| user12@gatech.edu | F  | Austin

`%` sign matches any string, including the empty string.<br>
`_` matches any single character.
* Both can be used as a wildcard prefix/suffix on any string (used as suffix above)

### Sorting

Another practical consideration is the ability to sort your result sets. Think of how a phonebook is sorted by LastNames.

Let's say our objective is to *find data about RegularUsers who are males. Sort data by ascending CurrentCity*:

Given:

**RegularUser**

| Email | Sex | CurrentCity | HomeTown
| --- | --- | --- | --- |
| user1@gatech.edu | M | Seattle | Atlanta
| user3@gatech.edu | M | San Diego | Portland
| user4@gatech.edu | M | San Francisco | Atlanta
| user6@gatech.edu | F | San Diego | Atlanta
| user9@gatech.edu | F | Las Vegas | Atlanta
| user12@gatech.edu | M | College Park | Austin

SQL:

`SELECT Email, Sex, CurrentCity, HomeTown
FROM RegularUser
WHERE Sex='M'
ORDER BY CurrentCity ASC`

Result:

| Email | Sex | CurrentCity | HomeTown
| --- | --- | --- | --- |
| user12@gatech.edu | M | College Park | Austin
| user3@gatech.edu | M | San Diego | Portland
| user4@gatech.edu | M | San Francisco | Atlanta
| user1@gatech.edu | M | Seattle | Atlanta

We can sort ascending using `ORDER BY col_n ASC`. Ascending is the default.<br>
Descending using `ORDER BY col_n DESC`<br>
And mutiple columns ascending and/or descending `ORDER BY col_1, col_2, col_n DESC`

### Set Operations

#### Union

Let's say our objective is to *find all CurrentCity and HomeTowns of RegularUser without duplicates*:

Given:

**RegularUser**

| Email | Sex | CurrentCity | HomeTown
| --- | --- | --- | --- |
| user1@gatech.edu | M | Seattle | Atlanta
| user3@gatech.edu | M | San Diego | Portland
| user4@gatech.edu | M | San Francisco | Atlanta
| user6@gatech.edu | F | San Diego | Atlanta
| user9@gatech.edu | F | Las Vegas | Atlanta
| user12@gatech.edu | M | College Park | Austin

SQL:

`SELECT CurrentCity
FROM RegularUser
UNION
SELECT HomeTown
FROM RegularUser`

Result:

|  |
| --- |
| Seattle | 
| San Diego | 
| San Francisco |
| Las Vegas |
| College Park | 
| Atlanta |
| Portland |
| Austin |

Note that set operators including `UNION` always return sets - ie there are never duplicates in the result set.<br>
If we wanted to preserve duplicates in our result set, we can use `UNION ALL`:

SQL:

`SELECT CurrentCity
FROM RegularUser
UNION ALL
SELECT HomeTown
FROM RegularUser`

Result:

|  |
| --- |
| Seattle | 
| San Diego | 
| San Francisco |
| San Diego | 
| Las Vegas |
| College Park | 
| Atlanta |
| Portland |
| Atlanta |
| Atlanta |
| Atlanta |
| Austin |

Note the duplicate values returned in the result set.

#### Intersect

Let's say our objective is to *find all cities that are a CurrentCity for some RegularUser and a HomeTown of some RegularUser without duplicates*:

*Note:* `RegularUser.CurrentCity` does not neccesarily have to equal `RegularUser.HomeTown`

Given:

**RegularUser**

| Email  | CurrentCity | HomeTown
| ---  | --- | --- |
| user1@gatech.edu  | Seattle | Atlanta
| user2@gatech.edu  | Austin | Austin
| user3@gatech.edu  | San Diego | Portland
| user4@gatech.edu  | San Francisco | Atlanta
| user8@gatech.edu  | College Park | Atlanta
| user12@gatech.edu | College Park | Austin
| user14@gatech.edu | Austin | San Francisco

SQL:

`SELECT CurrentCity
FROM RegularUser
INTERSECT
SELECT HomeTown
FROM RegularUser`

Result:

|  |
| --- |
| Austin |
| San Francisco |

Note that the only tuples returned are those that appear in both the `CurrentCity` and `HomeTown` columns.<br>
To preserve duplicates, use `INTERSECT ALL` statement.

SQL:

`SELECT CurrentCity
FROM RegularUser
INTERSECT ALL
SELECT HomeTown
FROM RegularUser`

Result:

|  |
| --- |
| Austin |
| Austin |
| San Francisco |

Austin appears twice in both columns (total of 4 times) so it appears twice in the result set.

#### Except

Let's say our objective is to *find all cities that are a CurrentCity for some RegularUser but exclude those that are HomeTowns without duplicates*:

Given:

**RegularUser**

| Email  | CurrentCity | HomeTown
| ---  | --- | --- |
| user1@gatech.edu  | Seattle | Atlanta
| user2@gatech.edu  | Austin | Austin
| user3@gatech.edu  | San Diego | Portland
| user4@gatech.edu  | San Francisco | Atlanta
| user8@gatech.edu  | College Park | Atlanta
| user12@gatech.edu | College Park | Austin
| user14@gatech.edu | Austin | San Francisco
| user17@gatech.edu | Austin | Atlanta

SQL:

`SELECT CurrentCity
FROM RegularUser
EXCEPT
SELECT HomeTown
FROM RegularUser`

Result:

|  |
| --- |
| Seattle |
| San Diego |
| College Park |

Note that the only tuples returned are those that appear in the `CurrentCity` and not the `HomeTown` column.<br>
To preserve duplicates and include cities that appear more times as `CurrentCity`s then as `HomeTown`a, use `EXCEPT ALL` statement:

SQL:

`SELECT CurrentCity
FROM RegularUser
EXCEPT ALL
SELECT HomeTown
FROM RegularUser`

Result:

|  |
| --- |
| Seattle |
| San Diego |
| College Park |
| College Park |
| Austin |

Note that since College Park appears twice in the `CurrentCity` column and zero times in the `HomeTown` column, it appears twice in our result set. Similarly, since Austin appears three times in the `CurrentCity` column and twice in the `HomeTown` column, it appears once in our result set.

### Built-in Functions count, max, min, avg

#### count

Let's say our objective is to *count the number of RegularUser*:

Given:

**RegularUser**

| Email | BirthYear | Sex
| --- | --- | --- |
| user1@gatech.edu | 1985 | M |
| user2@gatech.edu | 1969 | M |
| user3@gatech.edu | 1967 | M |
| user4@gatech.edu | 1988 | M |
| user6@gatech.edu | 1988 | F |
| user8@gatech.edu | 1968 | M |
| user9@gatech.edu | 1988 | F |

SQL:

`SELECT COUNT(*)
FROM RegularUser`

Result:

|  |
| --- |
| 7 |

#### max

Let's say our objective is to *find the Email and BirthYear for the youngest female RegularUser*:

SQL:

`SELECT Email, max(BirthYear)
FROM RegularUser
WHERE Sex='F'`

Result:

| Email | BirthYear |
| --- | --- |
| user6@gatech.edu | 1988 | F |
| user9@gatech.edu | 1988 | F |

#### min

Let's say our objective is to *find the Email and BirthYear for the oldest male RegularUser*:

SQL:

`SELECT Email, min(BirthYear)
FROM RegularUser
WHERE Sex='M'`

Result:

| Email | BirthYear |
| --- | --- |
| user3@gatech.edu | 1967 |

#### avg

Let's say our objective is to *find the avg age of male RegularUser*:

SQL:

`SELECT avg(BirthYear)
FROM RegularUser
WHERE Sex='M'`

Result:

|  |
| --- |
| 1975.4 |

So since the `avg(BirthYear)` is 1975.4, the average age is $2019 - 1975.4 = 43.6$ years for male `RegularUser`.

### Group By

Let's say our objective is as follows:

*Group UserInterests on Email*<br>
*For each group, return the Email, the number of Interests, and the average SinceAge for each group.*<br>
*Sort the result by ascending number of interests*

Given:

** UserInterests **

| Email | Interest | SinceAge
| --- | --- | --- |
| user1@gatech.edu | Music | 10 |
| user1@gatech.edu | Reading | 5 |
| user1@gatech.edu | Tennis | 14 |
| user2@gatech.edu | Blogging | 13 |
| user2@gatech.edu | Meditation | 21 |
| user2@gatech.edu | Surfing | 19 |
| user4@gatech.edu | DIY | 18 |
| user2@gatech.edu | Swimming | 1 |
| user2@gatech.edu | Tennis | 12 |

SQL:

`SELECT Email, count(*) as NumInterests, avg(SinceAge) AvgAge
FROM UserInterests
GROUP BY Email
ORDER BY NumInterests ASC;`

Result:

| Email | NumInterests | AvgAge
| --- | --- | --- |
| user4@gatech.edu | 1 | 18 |
| user1@gatech.edu | 3 | 9.67 |
| user2@gatech.edu | 5 | 13.2 |

#### Having

To apply conditions on our `GROUP BY` statement, we must use the `HAVING` keyword. The `HAVING` keyword has the same functionality as a `WHERE` clause, only applied to groups specified in a `GROUP BY` statement.

Let's say our objective from above changes to only include users with $>1$ NumInterests:

*Group UserInterests on Email*<br>
*For each group, return the Email, the number of Interests, and the average SinceAge for each group.*<br>
*Filter the result set to only include users with $>1$ NumInterests*<br>
*Sort the result by ascending number of interests*

SQL:

`SELECT Email, count(*) as NumInterests, avg(SinceAge) AvgAge
FROM UserInterests
GROUP BY Email
HAVING NumInterests > 1
ORDER BY NumInterests ASC;`

Result:

| Email | NumInterests | AvgAge
| --- | --- | --- |
| user1@gatech.edu | 3 | 9.67 |
| user2@gatech.edu | 5 | 13.2 |

### Nested Queries

The true power of SQL is in it's ability to perform nested queries, or queries that build upon intermittent result sets.

#### Nested Queries: In/Not In

Let's say our objective is to *find Email and Interests for RegularUser in Atlanta*:

Given:

**RegularUser**

| Email | BirthYear | Sex | HomeTown
| --- | --- | --- | --- |
| user1@gatech.edu | 1985 | M | Atlanta
| user2@gatech.edu | 1969 | M | Austin
| user3@gatech.edu | 1967 | M | Portland
| user4@gatech.edu | 1988 | M | Atlanta

** UserInterests **

| Email | Interest | SinceAge
| --- | --- | --- |
| user1@gatech.edu | Music | 10 |
| user1@gatech.edu | Reading | 5 |
| user1@gatech.edu | Tennis | 14 |
| user2@gatech.edu | Blogging | 13 |
| user3@gatech.edu | Music | 11 |
| user4@gatech.edu | DIY | 18 |

SQL:

`SELECT Email, Interest
FROM UserInterests
WHERE Email IN
(
    SELECT EMAIL
    FROM RegularUser
    WHERE HomeTown = 'Atlanta'
);`

Result:

The first intermittent result R1 returned is from the inner nested query:

R1:

| Email |
| --- |
| user1@gatech.edu |
| user4@gatech.edu |

Corresponding to `HomeTown = 'Atlanta'` in the RegularUser table.

The outer query uses R1 in it's `WHERE` statement to only pull UserInterests for `Email` specified in R1:

Final Result:

| Email | Interest |
| --- | --- |
| user1@gatech.edu | Music |
| user1@gatech.edu | Reading | 
| user1@gatech.edu | Tennis |
| user4@gatech.edu | DIY |


#### Nested Queries: $=,\neq,\leq,\geq, <, >$ Some/All

Let's say our objective is as follows:

*Find CurrentCity with atleast one RegularUser*<br>
*with a Salary that's higher than all Salaries of RegularUser*<br>
*with HomeTown Austin*

Given:

**RegularUser**

| Email | BirthYear | Sex | CurrentCity | HomeTown
| --- | --- | --- | --- |
| user1@gatech.edu | 1985 | M | Seattle | Atlanta
| user2@gatech.edu | 1969 | M | Austin | Austin
| user3@gatech.edu | 1967 | M | San Diego | Portland
| user6@gatech.edu | 1988 | F | San Diego | Atlanta
| user8@gatech.edu | 1968 | M | College Park | Atlanta
| user12@gatech.edu | 1974 | F | College Park | Austin

**YearSalary**

| BirthYear | Salary |
| --- | --- |
| 1985 | 27000 |
| 1969 | 43000 |
| 1967 | 45000 |
| 1968 | 44000 |
| 1988 | 24000 |
| 1986 | 26000 |
| 1974 | 38000 |

SQL:

`SELECT CurrentCity
FROM RegularUser R, YearSalary Y
WHERE R.BirthYear = Y.BirthYear
AND
Salary > ALL
(
    SELECT Salary
    FROM RegularUser R, YearSalary Y
    WHERE R.BirthYear = Y.BirthYear
    AND
    HomeTown = 'Austin'
);`

Result:

The first intermittent result R1 returned is from the inner nested query:

R1:

| Salary |
| --- |
| 43000 |
| 38000 |

Corresponding to `HomeTown = 'Austin'` in the RegularUser table.

The outer query uses R1 in it's `WHERE` statement to only pull `CurrentCity` for `Salary` greater than `ALL` salaries returned by R1:

Final Result:

| CurrentCity | 
| --- |
| San Diego |
| College Park |

#### Nested Queries - Correlated

Let's say our objective is to *find Email and BirthYear of RegularUser who have no Interests*:

Given:

**RegularUser**

| Email | BirthYear | Sex
| --- | --- | --- |
| user1@gatech.edu | 1985 | M |
| user2@gatech.edu | 1969 | M |
| user3@gatech.edu | 1967 | M |
| user4@gatech.edu | 1988 | M |
| user6@gatech.edu | 1988 | F |
| user8@gatech.edu | 1968 | M |
| user9@gatech.edu | 1988 | F |

** UserInterests **

| Email | Interest | SinceAge
| --- | --- | --- |
| user1@gatech.edu | Music | 10 |
| user2@gatech.edu | Blogging | 13 |
| user2@gatech.edu | Meditation | 21 |
| user3@gatech.edu | Music | 11 |
| user3@gatech.edu | Reading | 6 |
| user4@gatech.edu | DIY | 18 |

SQL:

`SELECT R.Email, BirthYear
FROM RegularUser R
WHERE NOT EXIST
(
    SELECT *
    FROM UserInterests U
    WHERE U.Email = R.Email
);`

Result:

There is no concept of intermittent results in correlated queries. If you notice above, our inner query join condition `WHERE U.Email = R.Email` references a tuple variable `R` that was declared outside of the scope of the inner query (in the outer query) - this is an essence what makes the two queries correlated.

***Think of it as a sub-query evaluated once for each row of the outer query.***

So think of a loop that loops over each `R.Email` and compares it against `U.Email`. If `R.Email = U.Email`, then the `NOT EXIST` condition specified in the `WHERE` clause evaluates false. To evaluate true, `R.Email != U.Email`.

Final Result:

| Email | BirthYear |
| --- | --- |
| user6@gatech.edu | 1988 |
| user8@gatech.edu | 1968 |
| user9@gatech.edu | 1988 |

Note that the `Email` in our final result do not appear anywhere in the UserInterests table.